In [129]:
import numpy as np
import functools as ft
import itertools
import os

In [184]:
qubit_number=4
Pauli=np.asarray([
        [[1,0],
        [0,1]],


        [[0,1],
        [1,0]],


        [[0,-1j],
        [1j,0]],


        [[1,0],
        [0,-1]]])


states=np.asarray([
    [1,1]/np.sqrt(2),
    [1,-1]/np.sqrt(2),
    [1,1j]/np.sqrt(2),
    [1,-1j]/np.sqrt(2),
    [1,0],
    [0,1],
])

In [185]:
projectors = []
perm = itertools.product(states, repeat=qubit_number)
for i in list(perm):
    projectors.append(np.outer(ft.reduce(np.kron, i),np.conjugate(np.transpose(ft.reduce(np.kron, i)))))
    
observables = []
perm = itertools.product(Pauli, repeat=qubit_number)
for i in list(perm):
    observables.append(ft.reduce(np.kron, i))

In [186]:
psi=np.zeros((len(observables[1:]),len(projectors)))
psi_aux=np.zeros((3**qubit_number,2**qubit_number,4**qubit_number-1))

for i in range(len(observables[1:])):
    for j in range(len(projectors)):
        psi[i][j]=np.trace(projectors[j]@observables[i+1])/(2**qubit_number)
        a=int(j%2+2*((int(j/6)%(2**(qubit_number-1)))))
        b=int(j/2)%3
        c=3*int(j/(6*2**(qubit_number-1)))
#         print(i,j,b+c,a,i)
        psi_aux[int(b+c)][a][i]=np.trace(projectors[j]@observables[i+1])/(2**qubit_number)

C:\Users\LauraMartins\AppData\Local\Temp\ipykernel_26864\3723427908.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  psi[i][j]=np.trace(projectors[j]@observables[i+1])/(2**qubit_number)
C:\Users\LauraMartins\AppData\Local\Temp\ipykernel_26864\3723427908.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  psi_aux[int(b+c)][a][i]=np.trace(projectors[j]@observables[i+1])/(2**qubit_number)


In [187]:
inv=np.linalg.inv(psi@np.transpose(psi))
            
working_dir = os.getcwd()
np.save(working_dir+'\SavedVariables\X_matrix'+str(qubit_number)+'trial', psi_aux)
np.save(working_dir+'\SavedVariables\invXtX_matrix'+str(qubit_number)+'trial', inv)

In [188]:
# m=np.load(working_dir+'\SavedVariables\X_matrix2.npy')
# print(np.round(psi_aux-m,3))